# PART 1: Data Scraping
## Connect to Reddit API

In [1]:
import praw

reddit = praw.Reddit(
    client_id="xn1EzLWHVbp2ZOuTiNYk-w",
    client_secret="rSRccp3N37yjKuQQWcPxW_SKSXe0SA",
    user_agent="<NanyangTechnologicalUniversity>-<information retrieval project>:<v1> (by /u/CitronResponsible811)",
    username="CitronResponsible811",
    password="Password123"
)

In [2]:
# Check if have read & write access
# Read access -> for requesting post & comments information
# Write access -> for requesting subreddit moderator list
reddit.user.me()

Redditor(name='CitronResponsible811')

## Documentation about reddit API crawling library
- Post docs: https://praw.readthedocs.io/en/stable/code_overview/models/submission.html#submission
- Comments docs: https://praw.readthedocs.io/en/stable/code_overview/models/comment.html

## Some errors encountered
- instances of the MoreComments class, which represent "load more comments" links on the Reddit website.

In [3]:
import csv
import emoji
import time
import os
from praw.models import MoreComments

data_dir = 'data'

def write_headers(subreddit_name):
    # Write header first
    with open(f'{os.path.join(data_dir, subreddit_name)}-posts.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'author',
            'author_flair_text',
            'clicked',
            'created_utc',
            'distinguished',
            'edited',
            'id',
            'is_original_content',
            'is_self',
            'link_flair_text',
            'locked',
            'name',
            'num_comments',
            'over_18',
            'permalink',
            'saved',
            'score',
            'selftext',
            'spoiler',
            'stickied',
            'subreddit_name',
            'title',
            'upvote_ratio',
            'url'
        ])
        
    with open(f'{os.path.join(data_dir, subreddit_name)}-comments.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'author',
            'body',
            'body_html',
            'created_utc',
            'distinguished',
            'edited',
            'id',
            'is_submitter',
            'link_id',
            'parent_id',
            'permalink',
            'saved',
            'score',
            'stickied',
            'subreddit_name',
            'subreddit_id'
        ])

In [4]:
def extract_all_post_comments_post_ids(mod_names, posts, subreddit_name, post_ids=[]):
    # Prep headers first
    write_headers(subreddit_name)
    
    for index, post in enumerate(posts, start=1):
        # Skip mod posts, usually about rules etc, not helpful at all
        if post.author and post.author.name in mod_names:
            print(f"No: {index}, Skipped mod post...")
            time.sleep(60) # Sleep for 60 seconds to avoid rate limit
            continue
        # Only write unique posts & comments
        if post.id not in post_ids:
            post_row = [
                getattr(post, 'author', '[deleted]') or '[deleted]', # Some accounts are deleted
                getattr(post, 'author_flair_text', 'None') or 'None',
                getattr(post, 'clicked', None) or 'None',
                # post.comments, # No need instance
                getattr(post, 'created_utc', None) or 'None',
                getattr(post, 'distinguished', None) or 'None',
                getattr(post, 'edited', None) or 'None',
                getattr(post, 'id', None) or 'None',
                getattr(post, 'is_original_content', None) or 'None',
                getattr(post, 'is_self', None) or 'None',
                #post.link_flair_template_id,
                getattr(post, 'link_flair_text', None) or 'None',
                getattr(post, 'locked', None) or 'None',
                getattr(post, 'name', None) or 'None',
                getattr(post, 'num_comments', None) or 'None',
                getattr(post, 'over_18', None) or 'None',
                getattr(post, 'permalink', None) or 'None',
                # post.poll_data,
                getattr(post, 'saved', None) or 'None',
                getattr(post, 'score', None) or 'None',
                getattr(post, 'selftext', None) or 'None',
                getattr(post, 'spoiler', None) or 'None',
                getattr(post, 'stickied', None) or 'None',
                subreddit_name,
                getattr(post, 'title', None) or 'None',
                getattr(post, 'upvote_ratio', None) or 'None',
                getattr(post, 'url', None) or 'None'
            ]
            print(f"No: {index}, Title: {subreddit_name}:{post.title}, PostID: {post.id}")
            
            with open(f'{os.path.join(data_dir, subreddit_name)}-posts.csv', 'a', newline='', encoding='utf-8') as f:
                # Write to file
                writer = csv.writer(f)
                writer.writerow(post_row)
                
            time.sleep(60) # Sleep for 60 seconds to avoid rate limit
    
            # Write post comments too
            post = reddit.submission(id=post_row[6]) # post.id
            
            # post.comments.replace_more(limit=None)  # Fetch all comments replies
            # for comment in post.comments.list():
            #     # Skip comments that are deleted
            #     if comment.body == '[deleted]':
            #         continue
            # If you want to fetch all comments, comment out skipping of MoreComments
            
            for comment in post.comments:
                if isinstance(comment, MoreComments):
                    # Skip lower level comments
                    continue
                # Skip mod comments, usually about rules
                if comment.author and comment.author.name in mod_names:
                    continue
                # Skip comments that are deleted
                if comment.body == '[deleted]' or comment.body == '[removed]':
                    continue

                comment_row = [
                    getattr(comment, 'author', '[deleted]') or '[deleted]', # Some accounts are deleted
                    emoji.demojize(getattr(comment, 'body', 'None')),
                    emoji.demojize(getattr(comment, 'body_html', 'None')),
                    getattr(comment, 'created_utc', None) or 'None',
                    getattr(comment, 'distinguished', None) or 'None',
                    getattr(comment, 'edited', None) or 'None',
                    getattr(comment, 'id', None) or 'None',
                    getattr(comment, 'is_submitter', None) or 'None',
                    getattr(comment, 'link_id', None) or 'None',
                    getattr(comment, 'parent_id', None) or 'None',
                    getattr(comment, 'permalink', None) or 'None',
                    # comment.replies, # No need instance, dont need replies to top level comments anyways
                    getattr(comment, 'saved', None) or 'None',
                    getattr(comment, 'score', None), # Downvotes - Upvotes
                    getattr(comment, 'stickied', None) or 'None',
                    # comment.submission, # No need instance, no need instance of user's posts
                    subreddit_name,
                    getattr(comment, 'subreddit_id', None) or None
                ]
    
                with open(f'{os.path.join(data_dir, subreddit_name)}-comments.csv', 'a', newline='', encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow(comment_row)
                    
            post_ids.append(post.id)
    return post_ids

# Get subreddit mod ids
def get_mod_list(subreddit_name):
    mod_names = []
    for moderator in reddit.subreddit(subreddit_name).moderator():
        mod_names.append(moderator.name)
        
    return mod_names

In [ ]:
# Start extracting
## Specialized subreddits
specialized_subreddits = [
    # General electric car subreddits
    # "electricvehicles",
    # "electriccars",
    # "nzev",
    # "ElectricCarUK",
    # "evcharging",
    
    # Electric Car brands subreddits
    # "teslamotors", # Tesla 100% electric car brand
    "RealTesla",
    "BMWi3", # BMW I3 car model electric
    "leaf", # Nissan LEAF EV car model
    "BoltEV", # Chevrolet Bolt EV car model
    "F150Lightning", # Ford F150 Lightning EV car model
    "Taycan", # Porsche Taycan EV car model
    "KiaEV6", # Kia EV6 car model
]

for subreddit_name in specialized_subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.top(limit=100, time_filter='all')
    extract_all_post_comments_post_ids(get_mod_list(subreddit_name), posts, subreddit_name)

No: 1, Title: teslamotors:Tesla's Summon feature was very useful today..., PostID: 7srdiw
No: 2, Title: teslamotors:Elon Musk confirms secret Tesla ‘Carless Driver’ project, PostID: 7tvnx6
No: 3, Title: teslamotors:Tesla vs Bugatti, PostID: 7e2ze2
No: 4, Title: teslamotors:My grandpa just turned 91. This is his birthday present to himself., PostID: ma79ze
No: 5, Title: teslamotors:Autopilot saves my model 3 from an accident!, PostID: andk8y
No: 6, Title: teslamotors:Update to the previous post, PostID: 7p6z9u
No: 7, Title: teslamotors:DAY ONE WITH MY TESLA!, PostID: 7yjrqa
No: 8, Title: teslamotors:Saw this beaut in a parking lot today. Didn’t realize the owner was in it until after I walked past because of the tinted windows., PostID: 8iq8u9
No: 9, Title: teslamotors:Elon Burn Ouch 🤕, PostID: l7fw2f
No: 10, Title: teslamotors:All black everything 😎, PostID: 9ezo6i
No: 11, Title: teslamotors:We have a visitor at work today, PostID: 7xrqyn
No: 12, Title: teslamotors:Bill Nye flexing wit

## General subreddits with search filters

In [ ]:
general_subreddits = [
    "cars",
]

search_terms = [
    'ev',
]

for subreddit_name in general_subreddits:
    post_ids = []
    for search_term in search_terms:
        subreddit = reddit.subreddit(subreddit_name)
        search_results = subreddit.search(search_term, sort='relevance', limit=100)
        post_ids.extend(extract_all_post_comments_post_ids(get_mod_list(subreddit_name), search_results, f'{subreddit_name}-{search_term}', post_ids))
        post_ids = list(set(post_ids))

# PART 2: Get dataset summaries
## Summary of all different subreddit posts & comments

In [1]:
# Load all comments file, check unique word count
import os
import pandas as pd

data_dir = 'data2'

# Since same format, combine all comments csv into a massive df
comment_main_df = pd.DataFrame()
post_main_df = pd.DataFrame()
for file in os.listdir(data_dir):
    df = pd.read_csv(f'{data_dir}/{file}')
    print(f"File: {file}, Rows: {len(df)}")
    if file.endswith("comments.csv"):
        comment_main_df = pd.concat([comment_main_df, df])
    elif file.endswith("posts.csv"):
        post_main_df = pd.concat([post_main_df, df])

File: BMWi3-comments.csv, Rows: 1259
File: BMWi3-posts.csv, Rows: 100
File: cars-diesel-comments.csv, Rows: 2578
File: cars-diesel-posts.csv, Rows: 94
File: cars-electric-comments.csv, Rows: 5579
File: cars-electric-posts.csv, Rows: 98
File: cars-gasoline-comments.csv, Rows: 2326
File: cars-gasoline-posts.csv, Rows: 99
File: cars-petrol-comments.csv, Rows: 2211
File: cars-petrol-posts.csv, Rows: 100
File: electriccars-comments.csv, Rows: 2337
File: electriccars-posts.csv, Rows: 100
File: ElectricCarUK-comments.csv, Rows: 99
File: ElectricCarUK-posts.csv, Rows: 100
File: electricvehicles-comments.csv, Rows: 6901
File: electricvehicles-posts.csv, Rows: 100
File: evcharging-comments.csv, Rows: 1432
File: evcharging-posts.csv, Rows: 100
File: nzev-comments.csv, Rows: 1565
File: nzev-posts.csv, Rows: 100
File: RealTesla-comments.csv, Rows: 10784
File: RealTesla-posts.csv, Rows: 100
File: TeslaModel3-comments.csv, Rows: 16375
File: TeslaModel3-posts.csv, Rows: 100
File: teslamotors-comments.

## Combined size

In [2]:
post_main_df.shape

(1291, 23)

In [3]:
comment_main_df.shape

(65256, 14)

## Checking for duplicated id & remove them

In [4]:
# Check for duplicates using id
post_main_df[post_main_df.duplicated(subset=['id'], keep='first')]

,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_id


In [6]:
comment_main_df[comment_main_df.duplicated(subset=['id'], keep='first')]

,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_id


In [ ]:
# Remove any duplicates detected
post_main_df.drop_duplicates(subset=['id'], keep='first')
comment_main_df.drop_duplicates(subset=['id'], keep='first')

## Checking for duplicated comments & post (Some might be crossposted, different id but same title but different comments)
- We decided to keep them as they are same post contents but different comments

In [15]:
post_main_df[post_main_df.duplicated(subset=['title'], keep=False)]

,author,author_flair_text,clicked,created_utc,distinguished,edited,id,is_original_content,is_self,link_flair_text,...,over_18,permalink,saved,score,selftext,spoiler,stickied,title,upvote_ratio,url
21,wewewawa,NaN,NaN,1.699695e+09,NaN,NaN,17sr2l7,NaN,NaN,NaN,...,NaN,/r/electriccars/comments/17sr2l7/if_you_bought...,NaN,126.0,NaN,NaN,NaN,If You Bought A Tesla One Year Ago It Has Lost...,0.82,https://jalopnik.com/if-you-bought-a-tesla-one...
58,ElectricVehicleNick,NaN,NaN,1.551115e+09,NaN,NaN,auno60,NaN,NaN,NaN,...,NaN,/r/ElectricCarUK/comments/auno60/nissan_leaf_p...,NaN,2.0,NaN,NaN,NaN,Nissan LEAF Plus first drive review: more of a...,1.00,https://electrek.co/2019/02/25/nissan-leaf-plu...
98,ElectricVehicleNick,NaN,NaN,1.551218e+09,NaN,NaN,av545a,NaN,NaN,NaN,...,NaN,/r/ElectricCarUK/comments/av545a/nissan_leaf_p...,NaN,1.0,NaN,NaN,NaN,Nissan LEAF Plus first drive review: more of a...,1.00,https://electrek.co/2019/02/25/nissan-leaf-plu...
34,manual_tranny,NaN,NaN,1.691671e+09,NaN,NaN,15nbggi,NaN,NaN,News,...,NaN,/r/electricvehicles/comments/15nbggi/disapprov...,NaN,2028.0,NaN,NaN,NaN,Disapproval of Elon Musk is top reason Tesla o...,0.91,https://electrek.co/2023/07/27/disapproval-elo...
5,gman1023,NaN,NaN,1.690573e+09,NaN,NaN,15c6xk9,NaN,NaN,HELP NEEDED,...,NaN,/r/RealTesla/comments/15c6xk9/disapproval_of_e...,NaN,5039.0,NaN,NaN,NaN,Disapproval of Elon Musk is top reason Tesla o...,0.95,https://electrek.co/2023/07/27/disapproval-elo...
75,TiltedWit,NaN,NaN,1.699650e+09,NaN,NaN,17sdqni,NaN,NaN,NaN,...,NaN,/r/RealTesla/comments/17sdqni/if_you_bought_a_...,NaN,1397.0,NaN,NaN,NaN,If You Bought A Tesla One Year Ago It Has Lost...,0.94,https://jalopnik.com/if-you-bought-a-tesla-one...


In [16]:
comment_main_df[comment_main_df.duplicated(subset=['body'], keep=False)]

,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_id
6,Congrats,"<div class=""md""><p>Congrats</p>\n</div>",1.648558e+09,NaN,NaN,i2k2yt1,NaN,t3_tqghan,t3_tqghan,/r/BMWi3/comments/tqghan/my_reward_for_finally...,NaN,3,NaN,t5_2y1ho
34,Congrats!,"<div class=""md""><p>Congrats!</p>\n</div>",1.708648e+09,NaN,NaN,krowqok,NaN,t3_1axmm77,t3_1axmm77,/r/BMWi3/comments/1axmm77/100k_miles/krowqok/,NaN,3,NaN,t5_2y1ho
70,Congrats!,"<div class=""md""><p>Congrats!</p>\n</div>",1.685016e+09,NaN,NaN,jljyc47,NaN,t3_13rc663,t3_13rc663,/r/BMWi3/comments/13rc663/picked_her_up_straig...,NaN,2,NaN,t5_2y1ho
92,Beautiful!,"<div class=""md""><p>Beautiful!</p>\n</div>",1.684768e+09,NaN,NaN,jl5xmy6,NaN,t3_13o1009,t3_13o1009,/r/BMWi3/comments/13o1009/picked_up_this_littl...,NaN,2,NaN,t5_2y1ho
105,Looks great!,"<div class=""md""><p>Looks great!</p>\n</div>",1.696521e+09,NaN,NaN,k3l86qm,NaN,t3_170lc98,t3_170lc98,/r/BMWi3/comments/170lc98/pulled_the_ev_trigge...,NaN,3,NaN,t5_2y1ho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11657,HAHAHAHA,"<div class=""md""><p>HAHAHAHA</p>\n</div>",1.565487e+09,NaN,NaN,ewkkc38,NaN,t3_cntie5,t3_cntie5,/r/teslamotors/comments/cntie5/when_youre_turn...,NaN,1,NaN,t5_2s3j5
11677,Amazing,"<div class=""md""><p>Amazing</p>\n</div>",1.569808e+09,NaN,NaN,f1y1r70,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,3,NaN,t5_2s3j5
11703,This is hilarious,"<div class=""md""><p>This is hilarious</p>\n</div>",1.569819e+09,NaN,NaN,f1ylkcu,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,1,NaN,t5_2s3j5
11713,Lel,"<div class=""md""><p>Lel</p>\n</div>",1.569838e+09,NaN,NaN,f1z7te8,NaN,t3_db1ttt,t3_db1ttt,/r/teslamotors/comments/db1ttt/getting_ready_f...,NaN,1,NaN,t5_2s3j5


## Save corpus to csv

In [11]:
with open(f'{os.path.join(data_dir, "all-posts.csv")}', 'w', newline='', encoding='utf-8') as f:
    post_main_df.to_csv(f, index=False)
    
with open(f'{os.path.join(data_dir, "all-comments.csv")}', 'w', newline='', encoding='utf-8') as f:
    comment_main_df.to_csv(f, index=False)